In [10]:
import pandas as pd
import requests
import os
import string

import csv

# Load the CSV file
csv_file_path = '/Users/shuai/Desktop/final_arts_info.csv'
df = pd.read_csv(csv_file_path)

csv_label = "art_label.csv"

# Specify a directory to save the images
# save_directory = '/Users/shuai/Desktop/images/'  # Change this to a directory where you have write access

# Create the directory if it doesn't exist
# if not os.path.exists(save_directory):
#     os.makedirs(save_directory)

# Function to clean and create a valid filename
def clean_filename(filename):
    valid_chars = "-_() %s%s" % (string.ascii_letters, string.digits)
    cleaned_filename = ''.join(c for c in filename if c in valid_chars)
    return cleaned_filename

with open(csv_label, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(
            [
                    "filename",
                    "year"
            ]
    )

# Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        url = row['link']
        year = str(row['year'])  # Convert year to string
        # Construct the filename using index and cleaned year
        filename = f"image_{index}_{clean_filename(year)}.jpg"  # Assuming the image format is jpg
        year_cleaned = clean_filename(year)[-4:].strip()

        print("filename: ", filename, "year: ", year_cleaned)

        writer.writerow(
            [
                filename,
                year_cleaned
            ]
        )
    

OSError: [Errno 30] Read-only file system: 'art_label.csv'

In [6]:
import csv
import json

def csv_to_jsonl(csv_file, jsonl_file):
    with open(csv_file, 'r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        with open(jsonl_file, 'w') as jsonlfile:
            for row in reader:
                json.dump(row, jsonlfile)
                jsonlfile.write('\n')

# Example usage:
csv_to_jsonl('final_art_label.csv', 'final_art_label.jsonl')


In [9]:
# 输入和输出文件路径
input_file_path = 'final_art_label.jsonl'
output_file_path = 'final_art_label_1.jsonl'

# 打开原始JSONL文件和新文件
with open(input_file_path, 'r', encoding='utf-8') as input_file, \
     open(output_file_path, 'w', encoding='utf-8') as output_file:
    # 逐行读取和处理每个JSON对象
    for line in input_file:
        # 将当前行转换为字典
        data = json.loads(line)
        # 修改属性名对应的值，添加前缀，并改变属性名
        if 'filename' in data:
            data['file_name'] = 'images/' + data.pop('filename')
            data['label'] = data.pop('year')
        # 将修改后的字典转换回JSON字符串，并写入新文件
        output_file.write(json.dumps(data) + '\n')
print("完成修改并生成新的JSONL文件。")

完成修改并生成新的JSONL文件。


In [10]:
import pandas as pd
import os
from PIL import Image
import io


def image_to_bytes(image_path):
    try:
        with Image.open(image_path) as img:
            byte_arr = io.BytesIO()
            img.save(byte_arr, format=img.format)
            return byte_arr.getvalue()
    except:
        pass
# 假设DataFrame中有一个名为'file_path'的列包含图片路径
metadata_df = pd.read_json('final_art_label_1.jsonl', lines=True)
metadata_df['image_data'] = metadata_df['file_name'].apply(image_to_bytes)


import pyarrow as pa
import pyarrow.parquet as pq
metadata_df.to_parquet('final_art_data.parquet', index=False)